In [4]:
import pandas as pd
import json

ModuleNotFoundError: No module named 'pandas'

In [4]:
with open("dummydata.json", "r") as file:
    data = json.load(file)

In [5]:
df = pd.json_normalize(data, record_path=["users"], sep="_")

In [6]:
print(df.head())

                         id                                   image  \
0  65b1234567890abcde123456         https://example.com/profile.jpg   
1  65b234567890abcded234567    https://example.com/profile_jane.jpg   
2  65b34567890abcdeef345678   https://example.com/profile_david.jpg   
3  65b4567890abcdeff4567890   https://example.com/profile_sarah.jpg   
4   65b67890abcde0000067890  https://example.com/profile_olivia.jpg   

               name                         email  age  gender       phone  \
0          John Doe          john.doe@example.com   32    MALE  9876543210   
1        Jane Smith        jane.smith@example.com   28  FEMALE  8765432109   
2         David Lee         david.lee@example.com   45    MALE  7654321098   
3       Sarah Brown       sarah.brown@example.com   25  FEMALE  6543210987   
4  Olivia Rodriguez  olivia.rodriguez@example.com   29  FEMALE  4321098765   

                     address           adhar         emailVerified  ...  \
0  123 Security Lane, City X 

In [7]:
from sklearn.preprocessing import LabelEncoder

In [8]:
df['workPlaceType'] = df['prevExperience'].apply(
    lambda x: x[0]['workPlace'] if x and isinstance(x, list) and len(x) > 0 else 'unknown'
)

In [9]:
df['totalExperienceYears'] = df['totalExperienceYears'].fillna(0)

In [10]:
df['averageRating'] = df['averageRating'].fillna(df['averageRating'].median())

In [11]:
label_encoder = LabelEncoder()
df['workPlaceType'] = label_encoder.fit_transform(df['workPlaceType'])
df['bestSuitedFor'] = label_encoder.fit_transform(df['bestSuitedFor'])

In [12]:
X = df[['totalExperienceYears', 'averageRating', 'workPlaceType']]
y = df['bestSuitedFor']

In [13]:
print(df[['id', 'totalExperienceYears', 'averageRating', 'workPlaceType', 'bestSuitedFor']].head())


                         id  totalExperienceYears  averageRating  \
0  65b1234567890abcde123456                   5.0           4.20   
1  65b234567890abcded234567                   3.0           4.44   
2  65b34567890abcdeef345678                   7.0           4.02   
3  65b4567890abcdeff4567890                   2.0           4.30   
4   65b67890abcde0000067890                   4.0           4.26   

   workPlaceType  bestSuitedFor  
0             46              0  
1             29              1  
2              4              2  
3             26              1  
4             44              1  


In [14]:
import xgboost as xgb
import joblib

In [15]:
model = xgb.XGBClassifier(objective='multi:softmax', num_class=3, random_state=42)
model.fit(X, y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None, num_class=3,
              num_parallel_tree=None, ...)

In [16]:
joblib.dump(model, "security_guard_model.pkl")
joblib.dump(label_encoder, "label_encoder.pkl")

['label_encoder.pkl']